#### 라이브러리 불러오기

In [1]:
import os
import platform

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

from sklearn.model_selection import train_test_split

# 회귀모델 평가 지표
from sklearn.metrics import mean_squared_error
# RMSE(Root Mean Squared Error) = np.sqrt(mse)
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

from sklearn.tree import DecisionTreeRegressor

2023-08-25 15:34:08.527564: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### 설정

In [2]:
# Check OS
os_name = platform.system()

# font_path는 사용할 폰트에 따라 변경하면 됨
font_path = 'c:/Windows/Fonts/malgun.ttf' if os_name=='Windows' \
    else '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_family = fm.FontProperties(fname=font_path).get_name()

# font 설정
plt.rcParams['font.family'] = font_family

# 그래프의 x,y축 레이블에 마이너스 값이 있을 때,
# 우리가 알아보기 쉽게 '-'로 표시되게 하는 옵션
plt.rcParams['axes.unicode_minus'] = False 

In [3]:
# 폰트캐시까지 삭제 후 한글폰트가 사용될 수 있도록 캐시 삭제
# --> 이렇게 해야 비로소 한글이 표현되는 경우가 많음
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = [font_family]

#### 평가함수 생성

In [4]:
def evaluation_func(test_y, pred_y, verbose=1):
    mse = mean_squared_error(test_y, pred_y)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_y, pred_y)
    mape = mean_absolute_percentage_error(test_y, pred_y)
    
    # 예측값이 0인 경우 대비 아주 작은값 설정
    epsilon = 1e-10
    abs_errors = np.abs(test_y, pred_y)
    percent_errors = (abs_errors / (test_y+epsilon)) * 100
    mape2 = np.mean(percent_errors)
    
    if verbose != 0:
        print(
            f'MSE: {mse:.6f}, RMSE: {rmse:.6f}, '
            f'MAE: {mae:.6f}, '
            f'MAPE: {mape:.6f}, MAPE2: {mape2:.6f}'
        )
        
    return mse, rmse, mae, mape, mape2

In [40]:
epsilon = 1e-10
test_y = test_y + epsilon

In [46]:
abs_error = np.abs(test_y, pred_mlps)

In [48]:
percent_error = (1 - (abs_error / test_y)) * 100

In [49]:
np.mean(percent_error)

0.0

In [35]:
zero_idx = percent_error[percent_error==0].index
test_y[zero_idx]

tm
2021-09-09    0.0
2021-09-24    0.0
Name: 발전량(kWh), dtype: float64

In [39]:
pred_mlps[zero_idx]

tm
2021-09-09    0.0
2021-09-24    0.0
dtype: float64

#### 데이터 불러오기

In [5]:
DATA_PATH       = os.path.join(os.getcwd(), 'data', '31-wind-power')
POWER_PATH      = os.path.join(DATA_PATH, 'train_energy.csv') 
WEATHER_PATH    = os.path.join(DATA_PATH, 'train_weather.csv')
TEST_PATH       = os.path.join(DATA_PATH, 'test.csv')

In [6]:
power_df = pd.read_csv(POWER_PATH, encoding='cp949')
weather_df = pd.read_csv(WEATHER_PATH)
test_df = pd.read_csv(TEST_PATH)

#### 데이터 확인

In [7]:
# merge할 power와 weather는 최소값과 최대값이 동일해 문제없는데,
# test 데이터를 포함하고 있기 때문에, 전처리 과정에서 빼고 테스트를 해야할듯함.
power_df.일자.min(), power_df.일자.max(), \
weather_df.tm.min(), weather_df.tm.max(), \
test_df.tm.min(), test_df.tm.max()

('2019-01-01',
 '2021-09-24',
 '2019-01-01',
 '2021-09-24',
 '2021-06-17',
 '2021-09-24')

#### 데이터 전처리

##### 일자 데이터 타입 변경: object -> datetime
key로 사용해야 하고, 또 다양한 일자정보를 활용하기 위해 변환

In [8]:
power_df['일자'] = pd.to_datetime(power_df['일자'])
weather_df['tm'] = pd.to_datetime(weather_df['tm'])
test_df['tm'] = pd.to_datetime(test_df['tm'])

##### Merge: weather <- power

In [9]:
# inner: weather에 없는 power의 row는 삭제
train_df = pd.merge(weather_df, power_df, left_on='tm', right_on='일자', how='inner')

##### 중복 일자데이터 제거

In [10]:
# 중복된 일자정보 제거
# 아래와 같이 할 수 있지만 아직 object type로 남아있는 데이터가 있어 눈으로 확인
# train_dt_cols = train_df.select_dtypes(include=['datetime64']) 

# train_dt_cols = ['tm', '일자']
# test_dt_cols = ['tm', '일자_tm', '일자']

# key로 사용할 'tm'을 제외하고 모두 삭제
train_df.drop('일자', axis=1, inplace=True)
test_df.drop(columns=['일자', '일자_tm'], axis=1, inplace=True)

##### test에 있는 컬럼만 남기고 제거
학습에 필요한 데이터는 테스트에서 입력으로 제공할 수 있는 데이터만 필요함

In [11]:
train_df = train_df[test_df.columns]

##### 컬럼의 고유값이 1개인 컬럼 제거

In [12]:
# 이 경우 트레인에는 유니크한 값이 1개뿐이지만,
# 테스트에서는 유니크한 값이 2개 이상인 컬럼도 제거,
# --> 트레인 시 모델에 영향없음 = 테스트에도 영향없음

train_n_unique_per_column = train_df.nunique()
# 이건 그냥 확인차
test_n_unique_per_column = test_df.nunique()

In [13]:
train_n_unique_per_column[train_n_unique_per_column == 1].index

Index(['stnId', 'stnNm', '발전기명(WTG)', '발전기(Serial)', '발전용량(kW)', '위치'], dtype='object')

In [14]:
test_n_unique_per_column[test_n_unique_per_column == 1].index

Index(['stnId', 'stnNm', '발전기명(WTG)', '발전기(Serial)', '발전용량(kW)', '위치'], dtype='object')

In [15]:
# 제거
unique_cols = train_df.columns[train_df.nunique() == 1]
train_df.drop(columns=unique_cols, axis=1, inplace=True)
test_df.drop(columns=unique_cols, axis=1, inplace=True)

##### Profile Report

In [16]:
# 이 셀 자체를 가급적 사용하지 말자

# 시간이 많이 소요되기 때문에 가급적 한번만 하고,
# profile.to_notebook_iframe() -> 이 명령은 사용 금지(ipynb파일 크기가 늘어남)
# profile = ProfileReport(train_df)
# profile.to_file(os.path.join(DATA_PATH, 'train_profile.html'))

##### 인덱스 변경(날짜로)

In [17]:
# 이미 위에서 했으면 더 편했을 수도
train_df = train_df.set_index('tm')
test_df = test_df.set_index('tm')

##### 결측치 처리

In [18]:
train_df.isna().sum().sort_values(ascending=False)

avgLmac           25
sumSmlEv          17
sumLrgEv          17
maxWd             12
avgCm30Te         10
avgCm20Te         10
avgCm10Te         10
avgCm5Te          10
avgTa              5
avgRhm             5
sumGsr             5
hr1MaxIcsr         5
hr1MaxIcsrHrmt     5
avgPa              5
avgPv              5
avgPs              5
avgTd              5
avgTca             3
sumSsHr            2
ssDur              1
maxTa              1
maxTaHrmt          0
minTaHrmt          0
발전량(kWh)           0
최저풍속(m-s)          0
minRhmHrmt         0
maxInsWsWd         0
평균풍속(m-s)          0
최대풍속(m-s)          0
최소발전량(kW)          0
평균발전량(kW)          0
최대발전량(kW)          0
이용률(백분율)           0
maxInsWs           0
avgTs              0
maxInsWsHrmt       0
minTg              0
minRhm             0
maxWs              0
maxWsWd            0
maxWsHrmt          0
avgWs              0
minTa              0
minPsHrmt          0
minPs              0
maxPsHrmt          0
maxPs              0
hr24SumRws   

In [19]:
# NaN 처리
# 인덱스가 시계열 컬럼일 경우, 시계열성을 반영해 중간값 지정
train_df = train_df.interpolate()

In [20]:
train_df.isna().sum().sort_values(ascending=False)

avgTa             0
ssDur             0
hr1MaxIcsrHrmt    0
hr1MaxIcsr        0
sumGsr            0
avgTca            0
avgLmac           0
avgTs             0
minTg             0
avgCm5Te          0
avgCm10Te         0
avgCm20Te         0
avgCm30Te         0
sumLrgEv          0
sumSmlEv          0
발전량(kWh)          0
최저풍속(m-s)         0
평균풍속(m-s)         0
최대풍속(m-s)         0
최소발전량(kW)         0
평균발전량(kW)         0
최대발전량(kW)         0
이용률(백분율)          0
sumSsHr           0
avgPs             0
minTa             0
minPsHrmt         0
minTaHrmt         0
maxTa             0
maxTaHrmt         0
maxInsWs          0
maxInsWsWd        0
maxInsWsHrmt      0
maxWs             0
maxWsWd           0
maxWsHrmt         0
avgWs             0
hr24SumRws        0
maxWd             0
avgTd             0
minRhm            0
minRhmHrmt        0
avgRhm            0
avgPv             0
avgPa             0
maxPs             0
maxPsHrmt         0
minPs             0
가동율(백분율)          0
dtype: int64

##### 타겟 컬럼 분류

In [21]:
target_col = '발전량(kWh)'
train_x = train_df.drop(target_col, axis=1)
train_y = train_df[target_col]
test_x = test_df.drop(target_col, axis=1)
test_y = test_df[target_col]

#### 기본 모델링
NaN처리만 한 데이터(나중에 정규화 + 시계열 속성 추가해 다시 모델링함)

In [22]:
model_dtr = DecisionTreeRegressor().fit(train_x, train_y)
pred_dtr = model_dtr.predict(test_x)
_ = evaluation_func(test_y, pred_dtr)

MSE: 0.000000, RMSE: 0.000000, MAE: 0.000000, MAPE: 0.000000, MAPE2: 98.000000


* CNN

In [23]:
# 모델생성
model_mlp = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(train_x.shape[1:])),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    # 회귀모델에서는 출력층에 활성화함수를 사용하지 않음
    Dense(1)
])
model_mlp.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mae', 'mape']
)
model_mlp.summary()

2023-08-25 15:34:13.217605: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22787 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:5e:00.0, compute capability: 7.5
2023-08-25 15:34:13.218421: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22272 MB memory:  -> device: 1, name: NVIDIA TITAN RTX, pci bus id: 0000:af:00.0, compute capability: 7.5


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               6272      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 39,425
Trainable params: 39,425
Non-trainable params: 0
_________________________________________________________________


In [24]:
model_mlp.fit(
    train_x, train_y, 
    epochs=100, verbose=0, validation_split=0.2
)

2023-08-25 15:34:15.560594: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fc54801c1c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-25 15:34:15.560670: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA TITAN RTX, Compute Capability 7.5
2023-08-25 15:34:15.560695: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (1): NVIDIA TITAN RTX, Compute Capability 7.5
2023-08-25 15:34:15.576954: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-25 15:34:15.806072: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-08-25 15:34:15.881654: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-25 15:34:15.996034: I ./tensorflow/compiler/jit/device_comp

In [25]:
model_mlp.evaluate(test_x, test_y, verbose=0)

[263745.6875, 442.40887451171875, 3448372480.0]

In [38]:
pred_mlp = model_mlp.predict(test_x)
pred_mlps = pd.Series(pred_mlp.flatten(), index=test_x.index)
_ = evaluation_func(test_y, pred_mlps)

4/4 [==============================] - 0s 3ms/step
MSE: 263745.725198, RMSE: 513.561803, MAE: 442.408887, MAPE: 15530090469429084.000000, MAPE2: 98.000000


In [ ]:
r2_score(test_y, pred_mlp)

0.9985056069716298

##### train에 있는 test데이터 삭제

In [ ]:
# 같은 날짜의 target 컬럼이 같은지 비교
temp_df = pd.DataFrame({'test': test_y, 'train': train_y})

In [ ]:
_temp_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   발전량(kWh)  120 non-null    float64
dtypes: float64(1)
memory usage: 1.1 KB
